# Finding Period and Initial Best Fit

In [1]:
#Importing and notebook setup
%matplotlib inline
import os

#For Plotting
import matplotlib.pyplot as plt

#Numbers and Functions
import numpy as np
#Used to open excel file of data
import pandas as pd
#For Maximum Likelihood
from scipy import optimize
#For Periodogram
from gatspy.periodic import LombScargleFast

#Making Tables
from tabulate import tabulate

#for Radvel
import matplotlib
import pylab as pl

import corner

#importing radvel
import radvel
import radvel.plotting

matplotlib.rcParams['font.size'] = 14

ImportError: No module named celerite

Define Periodogram Function

In [ ]:
def periodogram(datax, datay, min_, max_, nyquist):
    #finding periodogram
    model = LombScargleFast().fit(datax, datay)
    period, power = model.periodogram_auto(nyquist_factor=nyquist) # Default 50

    #Plotting
    plt.figure
    plt.plot(period,power)
    plt.ylabel('Power')
    plt.xlabel('Period')# days
    plt.xscale('log')
    #used bottom line to zoom in periodogram
#     plt.xlim(min_-1,max_+10)

    # set range and find period
    model.optimizer.period_range=(min_, max_)
    period = model.best_period
    print("period = {0}".format(period))
    return period

Function for RadVel and Residuals

Define a Plotting Function to Display the Data, Model, and Residuals

In [ ]:
#used for plotting radvel plots
def plot_results(like):
    fig = pl.figure(figsize=(12,4))
    fig = pl.gcf()
    pl.errorbar(
        like.x, like.model(t)+like.residuals(), 
        yerr=like.yerr, fmt='o'
        )
    pl.plot(ti, like.model(ti))
    fig.set_tight_layout(True)
    pl.xlabel('Time')
    pl.ylabel('RV')
    pl.draw()

Define RMS Function

In [ ]:
#getting RMS
def RMS (Residuals):
    RMS = np.std(Residuals)
    return RMS

Using Radvel to Find Model Likelihood

In [ ]:
#Found in Radvel.Fitting
def model_likelihood(post,show=False):
    num_planet = post.likelihood.model.num_planets
    ndata = len(post.likelihood.y)
    nfree = len(post.get_vary_params())
    rms = np.std(post.likelihood.residuals())
    logprob = post.logprob()
    chi = np.sum((post.likelihood.residuals()/post.likelihood.errorbars())**2)
    chi_red = chi / (ndata - nfree)
    bic = post.bic()
    if show:
        print "N_free = %d" % nfree
        print "RMS = %4.2f" % np.std(post.likelihood.residuals())
        print "logprob (jitter fixed) = %4.2f" % post.logprob() #radvel.likelihood
        print "chi (jitter fixed) = %4.2f" % chi
        print "chi_red (jitter fixed) = %4.2f" % chi_red
        print "BIC (jitter fixed) = %4.2f" % post.bic()
        
    model_likelihood = [num_planet, ndata, nfree, rms, logprob, chi , chi_red, bic]
    return model_likelihood

My Own Method of Modeling Data

In [ ]:
#My own method of plotting and finding likelihood
def residuals(num_planet,data_x, data_y,data_err,amplitude,period,phase_shift,vertical_shift):
    # Target function
    fitfunc = lambda p, x: p[0]*np.cos(2*np.pi/p[1]*x+p[2]) + p[3]*x
    #Distance to the target function
    errfunc = lambda p, x, y: fitfunc(p, x) - y 
    
    # Initial guess for the first set's parameters
    p0 = [amplitude, period, phase_shift, vertical_shift]
    p1, success = optimize.leastsq(errfunc, p0[:], args=(data_x, data_y))
    
    print 'Amplitude: ', np.absolute(p1[0])
    print 'Period: ', p1[1]
    print 'Phase Shift: ', p1[2]
    print 'Vertical-Shift: ',p1[3]
    period = p1[1]
    
    #Finding Residuals
    Residuals = data_y - fitfunc(p1,data_x)
    
    #Finding Likelihood
    ndata = len(data_x)
    nfree = 4.
    RMS = np.std(Residuals)
    logprob = np.log(np.sum(Residuals))
    chi = np.sum(((Residuals)/data_err)**2)
    chi_red = chi / (ndata - nfree)
#     bic = -2.0 * self.logprob() + len(self.likelihood.get_vary_params()) + np.log(len(self.likelihood.y))
    bic = np.log(ndata)+nfree +-2.*logprob
    #chisquared = sum(Residuals**2/(fitfunc(p1,data_x)))
    print "RMS of Residuals: ", RMS
    print "RMS", RMS
    print 'Chi Square', chi
    likelihood = [num_planet, ndata, nfree, RMS, logprob, chi , chi_red, bic]
    
    #Plotting Data with Fit
    time = np.linspace(data_x.min(), data_x.max(), 1000*len(data_x))
    plt.plot(data_x, data_y, "go", time, fitfunc(p1, time),"b-",alpha=0.5)
    plt.title('HD75732')
    plt.xlabel('Time [JD-2.44e6]')
    plt.ylabel('Mean Velocity[m/s]')
    plt.legend(('Data', 'Fit'))
    plt.xlim(data_x[0]-5,data_x[-1]+5)
    ax1 = plt.axes()
    plt.show()
    
    #plotting Residuals
    #plt.plot(data_x, Residuals, "ro", time, fitfunc(p2, time),"b-")
    plt.plot(data_x, Residuals, "ro")
    plt.xlim(data_x[0]-5,data_x[-1]+5)
    plt.ylim(-max(data_y)-10, max(data_y)+10)
    plt.title('Residuals')
    plt.xlabel('Time [JD-2.44e6]')
    plt.ylabel('Mean Velocity[m/s]')
    plt.legend(('Residuals', 'Fit'))
    ax2 = plt.axes()
    
    
    return Residuals, period, likelihood

Importing Data

In [ ]:
#Data import
#here data_rj includes both rk data and rj data
data= pd.read_csv('koi265_rvs.csv')

#data from different telescopes

dataOne = data[0:8]
dataTwo = data[8:13]
dataThree = data[13:27]
dataFour = data[27:38]
dataFive = data[38:]

# data = dataOne.append(dataTwo.append(dataThree.append(dataFour)))

#Naming Variables from data
t = np.array(data.jd)
vel = np.array(data.mnvel)
errvel = np.array(data.errvel)
# tel = np.array(data.tel)

#time vector for best fit
ti = np.linspace(t.min(),t.max(),num=len(t)* 10000)
# print data

# Finding Period in Data (1st Planet)

Initializing Radvel

In [ ]:
#nyquist - min period for data
nyquist = .5
#finding first planet period
period_guess = 3.56760613628 
maxPer = period_guess*1.25
minPer = period_guess*.75
# maxPer = (t.max()-t.min())/2.

# minPer = 1.5
# period1 = periodogram(t,vel,minPer,maxPer,nyquist)
period1 = 3.56760613628 

In [ ]:
nplanets = 1 # number of planets
# instnames = ['k', 'j']    # list of instrument names. Can be whatever you like but should match 'tel' column in the input file.
# ntels = len(instnames)       # number of instruments with unique velocity zero-points

def initialize_model():
    time_base = 0
    params = radvel.RVParameters(nplanets,basis='per tc secosw sesinw logk')
    
    #1st Planet
    params['per1'] = period1    # period of 1st planet
    params['tc1'] = 1300.0  # time of inferior conjunction of 1st planet
    params['secosw1'] = 0.01 
    params['sesinw1'] =  0.01 
    params['logk1'] =  np.log(8)   # velocity semi-amplitude for 1st planet  
    

    mod = radvel.RVModel(params, time_base=time_base)
    mod.params['dvdt'] = -.0001       # slope
    mod.params['curv'] = 0.0        # curvature  
    return mod

In [ ]:
mod = initialize_model()
like = radvel.likelihood.RVLikelihood(mod, t, vel, errvel)
like.params['gamma'] = 1.0
like.params['jit'] = 1.0

# like.params['gamma_k'] = 0.0
# like.params['gamma_j'] = 1.0
# like.params['jit_k'] = 2.6
# like.params['jit_j'] = 2.6

Defining Variables that are Going to Vary.

In [ ]:
# like.vary['curv'] = True
# like.vary['dvdt'] = True
# like.vary['per1'] = False
# like.vary['logk1'] = True
# like.vary['secosw1'] = False
# like.vary['sesinw1'] = False
#like.vary['tc1'] = False

Plotting Radvels Initial Likelihood

In [ ]:
pl.figure()
plot_results(like)
# pl.xlim(16460,16550)
# radvel.plotting.rv_multipanel_plot(post, epoch=2440000)

Maximize the Likelihood and Print the Updated Posterior Object

In [ ]:
post = radvel.posterior.Posterior(like)

post.priors += [radvel.prior.Gaussian('per1',like.params['per1'],.00025*like.params['per1'])]
post.priors += [radvel.prior.Gaussian('secosw1',like.params['secosw1'],.2)]
post.priors += [radvel.prior.Gaussian('sesinw1',like.params['sesinw1'],.2)]



post.priors += [radvel.prior.EccentricityPrior(nplanets)] #change for # of planets
post.priors += [radvel.prior.PositiveKPrior( nplanets )]

post.priors += [radvel.prior.Gaussian( 'jit', np.log(1), np.log(15))]
#post.priors += [radvel.prior.Gaussian( 'jit_k', np.log(1), np.log(15))]
#post.priors += [radvel.prior.Gaussian( 'jit_j', np.log(1), np.log(15))]

print post

In [ ]:
res  = optimize.minimize(
    post.neglogprob_array,     # objective function is negative log likelihood
    post.get_vary_params(),    # initial variable parameters
    method='Powell',           # Nelder-Mead also works
    )
#Get residuals
#orbel = []
#radvel.kepler.rv_drive(t)
plot_results(like)             # plot best fit model
print post
radvel.plotting.rv_multipanel_plot(post,epoch=2440000)

In [ ]:
# res  = optimize.minimize(
#     post.neglogprob_array,     # objective function is negative log likelihood
#     post.get_vary_params(),    # initial variable parameters
#     method='Powell',           # Nelder-Mead also works
#     )
# #Get residuals
# #orbel = []
# #radvel.kepler.rv_drive(t)
# plot_results(like)             # plot best fit model
# print post
# radvel.plotting.rv_multipanel_plot(post, telfmts=radvel.plotting.telfmts_default,epoch=2440000)

Finding the Model Likelihood, using RMS, Chi Squared, Log Probability, & BIC

In [ ]:
Residuals_Radvel_1 = post.likelihood.residuals()
RMS_Radvel_1 = RMS(Residuals_Radvel_1)
print 'Radvel RMS of Residuals: ', RMS_Radvel_1
print ' '
print ' '

one_planet= model_likelihood(post, True)

# Finding 2nd Planet

In [ ]:

#finding Second planet period

period_radvel_2  = periodogram(t,Residuals_Radvel_1,minPer,maxPer,nyquist)

In [ ]:
nplanets = 2
# instnames = ['k', 'j']    # list of instrument names. Can be whatever you like but should match 'tel' column in the input file.
# ntels = len(instnames)       # number of instruments with unique velocity zero-points

def initialize_model():
    time_base = 2420
    params = radvel.RVParameters(nplanets,basis='per tc secosw sesinw logk')
    
    #1st Planet
    params['per1'] = post.params['per1']    # period of 1st planet
    params['tc1'] = post.params['tc1']   # time of inferior conjunction of 1st planet
    params['secosw1'] = post.params['secosw1']
    params['sesinw1'] =  post.params['sesinw1']
    params['logk1'] =  post.params['logk1']   # velocity semi-amplitude for 1st planet
    
    #2nd planet    
    params['per2'] = period_radvel_2 
    params['tc2'] =  4268.95 + 1000   
    params['secosw2'] = 0.01  
    params['sesinw2'] = 0.01 
    params['logk2'] = np.log(RMS_Radvel_1)   
    

    mod = radvel.RVModel(params, time_base=time_base)
    mod.params['dvdt'] = 0.0         # slope
    mod.params['curv'] = 0.0        # curvature  
    return mod

In [ ]:
mod = initialize_model()
like = radvel.likelihood.RVLikelihood(mod, t, vel, errvel)
like.params['gamma'] = 1.0
like.params['jit'] = 2.6

like.vary['secosw1'] = False
like.vary['sesinw1'] = False
like.vary['secosw2'] = False
like.vary['sesinw2'] = False

like.vary['curv'] = True
like.vary['dvdt'] = True

pl.figure()
plot_results(like)

In [ ]:
post = radvel.posterior.Posterior(like)

post.priors += [radvel.prior.Gaussian('per1',like.params['per1'],.25*like.params['per1'])]
post.priors += [radvel.prior.Gaussian('per2',like.params['per2'],.25*like.params['per2'])]
post.priors += [radvel.prior.EccentricityPrior(nplanets)] #change for # of planets
post.priors += [radvel.prior.PositiveKPrior( nplanets )]

# post.priors += [radvel.prior.Gaussian( 'jit', np.log(1), np.log(15))]

print post

In [ ]:
res  = optimize.minimize(
    post.neglogprob_array,     # objective function is negative log likelihood
    post.get_vary_params(),    # initial variable parameters
    method='Powell',           # Nelder-Mead also works
    )

plot_results(like)             # plot best fit model
print post
radvel.plotting.rv_multipanel_plot(post, epoch=2440000)

In [ ]:
# res  = optimize.minimize(
#     post.neglogprob_array,     # objective function is negative log likelihood
#     post.get_vary_params(),    # initial variable parameters
#     method='Powell',           # Nelder-Mead also works
#     )
# #Get residuals
# #orbel = []
# #radvel.kepler.rv_drive(t)
# plot_results(like)             # plot best fit model
# print post
# radvel.plotting.rv_multipanel_plot(post, telfmts=radvel.plotting.telfmts_default,epoch=2440000)

In [ ]:
Residuals_Radvel_2 = post.likelihood.residuals()
RMS_Radvel_2 = RMS(Residuals_Radvel_2)
print 'Radvel RMS of Residuals: ', RMS_Radvel_2
print ' '
print ' '

#Model Likelihood
two_planet= model_likelihood(post, True)

In [2]:
#####MASSS##############
KOI_mass = (1.1070) * 1.989*10**30 #error =  0.1020 kg
KOI_radius =  1.3340 # error = 0.228 R_sun
KOI_V = 2.55 * (0.000210805) #m/s to Au/year
RV = 5
KOI_V = RV  * (0.000210805)
# print 1/(3.17098e-8/6.68459e-12)
# KOI_V = 5
print KOI_V
planet_per =3.56760613628 * 0.00273973 #day to year
planet_V = 2*np.pi*(KOI_mass/planet_per)**(1/3.) #Au/Year
planet_mass = ((KOI_mass*KOI_V)/planet_V)/(5.972*10**24) #earth masses

print planet_mass

0.001054025
1.01648522321e-09


In [3]:
print 0.012371665981-0.0116656533257

0.0007060126553


In [4]:
###DEnsity
planet_r = 0.978233804846 #-0.550446930839
planet_Size = (4./3)*np.pi*(planet_r)**3
planet_density = planet_mass/planet_Size
print planet_density

2.59229522753e-10


In [5]:
print 0.00690418026434 - 0.00301923495

0.00388494531434


In [6]:
0.005549853296839218

0.005549853296839218

In [15]:
# from radvel import utils 
K = 2.54561
P = 3.56760613628 *  0.00273973 # days to years
e = 0.0
K_0 = 28.4329
Mtotal = 1.1070 # + .1020

# Msini = radvel.utils.Msini(K,P,Mtotal, e, Msini_units='earth')
Msini = K / K_0 * np.sqrt(1.0 - e**2.0)*Mtotal**(2.0 / 3.0)*P**(1 / 3.0) *317.8
print Msini

# print 6.915952557999762 - 6.521280567953192

6.510053735916598


In [25]:
planet_r = 0.864556769278# -0.550446930839
planet_Size = (4./3)*np.pi*(planet_r)**3
planet_density = Msini/planet_Size
print planet_density * 5.513 #g/cm^3
print 0.003 * 5.513

13.258775278117021
0.016539


In [20]:
####
a = 13.53 * KOI_radius * 0.00464913034 # r sun to AU
print a

0.0839122464893


In [28]:
print 5.972 * 10**27/ (637.1*10**6)**3 /((4/3.)*np.pi)

5.51325873859
